# Gaussian Mixture Model

In [142]:
import torch
import numpy as np
from torch.nn import Softmax
from torch.autograd import Variable

import pyro
from pyro.infer.kl_qp import KL_QP
from pyro.distributions import categorical, diagnormal, dirichlet, gamma, normal
from pyro.util import ng_zeros, ng_ones

Let's start by modeling univariate real-valued data using a mixture of a fixed number of Gaussians with unknown weights `ps`, unknown means `mu`, and shared unknown variance `sigma`.

In [143]:
K = 2  # Fixed number of components.
z_values = np.array(range(K), dtype=np.int)
softmax = Softmax()

def model(data):
    # Global parameters.
    ps = pyro.sample('ps', dirichlet, alpha=ng_ones(K) * 0.5)

    # Per-component parameters with weak priors.
    mu = pyro.sample('mu', diagnormal, ng_zeros(K), ng_ones(K) * 100)
    sigma = pyro.sample('sigma', gamma, ng_ones(1), ng_ones(1))

    # Per-datum parameters.
    def local_model(i, datum):
        z = pyro.sample('z_{}'.format(i), categorical, ps, vs=z_values)[0]
        pyro.observe('x_{}'.format(i), datum, diagnormal, mu[z], sigma)
        return z

    return pyro.map_data('obs', data, local_model)

def guide(data):

    def local_guide(i, datum):
        p = pyro.param('p_{}'.format(i), ng_ones(K) / K)
        softmax_p = softmax(p.unsqueeze(0)).squeeze(0)  # Ugly.
        z = pyro.sample('z_{}'.format(i), categorical, softmax_p, vs=z_values)[0]
        return z

    return pyro.map_data('obs', data, local_guide)

In [144]:
optim_fct = pyro.optim(torch.optim.Adam, {'lr': 0.001})
inference = KL_QP(model, guide, optim_fct)

In [145]:
data = torch.Tensor([0, 1, 2, 7, 8, 9])

In [146]:
%pdb off
inference(data)

Automatic pdb calling has been turned OFF


AttributeError: 'torch.FloatTensor' object has no attribute 'log_pdf'